In [1]:
from sacremoses import MosesDetokenizer
import pandas as pd
import csv


In [2]:
data = pd.read_csv('questions.tsv', sep='\t', engine='python', on_bad_lines='warn')

Skipping line 443: '	' expected after '"'
Skipping line 444: '	' expected after '"'
Skipping line 445: '	' expected after '"'
Skipping line 446: '	' expected after '"'
Skipping line 507: '	' expected after '"'
Skipping line 508: '	' expected after '"'
Skipping line 712: '	' expected after '"'
Skipping line 713: '	' expected after '"'
Skipping line 714: '	' expected after '"'
Skipping line 715: '	' expected after '"'
Skipping line 737: '	' expected after '"'
Skipping line 822: '	' expected after '"'
Skipping line 823: '	' expected after '"'
Skipping line 824: '	' expected after '"'
Skipping line 825: '	' expected after '"'
Skipping line 826: '	' expected after '"'
Skipping line 827: '	' expected after '"'
Skipping line 1002: '	' expected after '"'
Skipping line 1048: '	' expected after '"'
Skipping line 1087: '	' expected after '"'
Skipping line 1151: '	' expected after '"'
Skipping line 1268: '	' expected after '"'
Skipping line 1269: '	' expected after '"'
Skipping line 1287: '	' expe

In [3]:
example = data['Sentence'][0]

In [4]:
md = MosesDetokenizer(lang='en')

In [5]:
md.detokenize(example.split())

'The nation ’ s largest gun - rights group is taking some Texans to task over their headline - generating demonstrations advocating the legal, open carrying of weapons.'

In [6]:
df = pd.DataFrame(data)
df

,Article_Id,Sentence_Id,Sentence,Span,Question,Span_Start_Position,Span_End_Position
0,1,1,The nation ’ s largest gun - rights group is t...,to task,"What does \""to task\"" mean?",13,15
1,1,1,The nation ’ s largest gun - rights group is t...,largest gun - rights group,What is this group called?,4,9
2,1,1,The nation ’ s largest gun - rights group is t...,gun - rights group,Which group?,5,9
3,1,1,The nation ’ s largest gun - rights group is t...,nation ’ s largest gun - rights group,Why don't you just come out and say the NRA?,1,9
4,1,2,Officials with the National Rifle Association ...,small number,How many people is a small number?,67,69
...,...,...,...,...,...,...,...
17778,1500,2,The $ 2 bill makes up just 3 percent of all pa...,3,Why is the $2 bill so uncommon?,7,8
17779,1500,3,"Now , it ' s about to get its time in the lime...","about to get its time in the limelight ,",How is it going to get time in the limelight?,5,14
17780,1500,3,"Now , it ' s about to get its time in the lime...","Delray Beach ,",Who is Delray Beach and why does he matter?,17,20
17781,1500,4,John Bennardo is crisscrossing the country to ...,"the story of the two and its "" magic """,Why are $2 bills seen as so much more desirable?,14,24


In [7]:
def moses_detokenize(row):
    return md.detokenize(row[2].split()).replace(u'\u201c', '"').replace(u'\u201d', '"')

In [8]:
detok_sent = df.apply(moses_detokenize, axis=1)
df['Sentence'] = detok_sent

In [9]:
filtered_data = df.filter(['Article_Id', 'Sentence', 'Question'], axis=1)

In [10]:
filtered_data

,Article_Id,Sentence,Question
0,1,The nation ’ s largest gun - rights group is t...,"What does \""to task\"" mean?"
1,1,The nation ’ s largest gun - rights group is t...,What is this group called?
2,1,The nation ’ s largest gun - rights group is t...,Which group?
3,1,The nation ’ s largest gun - rights group is t...,Why don't you just come out and say the NRA?
4,1,Officials with the National Rifle Association ...,How many people is a small number?
...,...,...,...
17778,1500,The $2 bill makes up just 3 percent of all pap...,Why is the $2 bill so uncommon?
17779,1500,"Now, it 's about to get its time in the limeli...",How is it going to get time in the limelight?
17780,1500,"Now, it 's about to get its time in the limeli...",Who is Delray Beach and why does he matter?
17781,1500,John Bennardo is crisscrossing the country to ...,Why are $2 bills seen as so much more desirable?


In [11]:
v_p1 = filtered_data.loc[filtered_data['Article_Id'] <= 100]
v_p2 = filtered_data.loc[(filtered_data['Article_Id'] > 1050) & (filtered_data['Article_Id'] <= 1100)]

In [12]:
validation = pd.concat([v_p1, v_p2])
validation

,Article_Id,Sentence,Question
0,1,The nation ’ s largest gun - rights group is t...,"What does \""to task\"" mean?"
1,1,The nation ’ s largest gun - rights group is t...,What is this group called?
2,1,The nation ’ s largest gun - rights group is t...,Which group?
3,1,The nation ’ s largest gun - rights group is t...,Why don't you just come out and say the NRA?
4,1,Officials with the National Rifle Association ...,How many people is a small number?
...,...,...,...
13152,1100,Opposition lawmakers had claimed the Democrats...,why would they want that?
13153,1100,Brozi 's court is due next week to consider th...,What espionage act did these four Greeks commit?
13154,1100,"In a secret vote, 53 deputies voted against li...",Were these deputies deliberately absent?
13155,1100,"In a secret vote, 53 deputies voted against li...",Why were so many absent?


In [13]:
t_p1 = filtered_data.loc[(filtered_data['Article_Id'] > 100) & (filtered_data['Article_Id'] <= 150)]
t_p2 = filtered_data.loc[(filtered_data['Article_Id'] > 500) & (filtered_data['Article_Id'] <= 550)]
t_p3 = filtered_data.loc[(filtered_data['Article_Id'] > 1100) & (filtered_data['Article_Id'] <= 1150)]

In [14]:
test = pd.concat([t_p1, t_p2, t_p3])
test

,Article_Id,Sentence,Question
1281,101,Olin Corp. said third - quarter net income ros...,What happened to the net income in the first a...
1282,101,Olin Corp. said third - quarter net income ros...,What was done differently in the third-quarter...
1283,101,Olin Corp. said third - quarter net income ros...,How much is 26% in dollars?
1284,101,Olin Corp. said third - quarter net income ros...,Third-quarter net income rose because the chem...
1285,101,"Net was $24 million, or $1. 15 a share, up fro...",What was done differently this year that was n...
...,...,...,...
13758,1150,Castro left the Dominican Republic in a pleasu...,"What is a \""pleasure boat\""?"
13759,1150,Castro left the Dominican Republic in a pleasu...,How many officials were arrested before Castro...
13760,1150,"The banker, believed to be in Miami, is charge...",How does what Castro paid compare to the rates...
13761,1150,"The banker, believed to be in Miami, is charge...",How do they know they are in Miami?


In [15]:
tr_p1 = filtered_data.loc[(filtered_data['Article_Id'] > 150) & (filtered_data['Article_Id'] <= 500)]
tr_p2 = filtered_data.loc[(filtered_data['Article_Id'] > 550) & (filtered_data['Article_Id'] <= 1050)]
tr_p3 = filtered_data.loc[(filtered_data['Article_Id'] > 1150)]

In [16]:
train = pd.concat([tr_p1, tr_p2, tr_p3])
train

,Article_Id,Sentence,Question
1889,151,Japan 's Daiwa Securities Co. named Masahiro D...,Why has Dozen chosen to be president?
1890,151,Japan 's Daiwa Securities Co. named Masahiro D...,What does Daiwa Securities Co do?
1891,151,Japan 's Daiwa Securities Co. named Masahiro D...,Why did they name Masahiro president?
1892,151,Japan 's Daiwa Securities Co. named Masahiro D...,What kind of company is this?
1893,151,"Mr. Dozen succeeds Sadakane Doi, who will beco...",Why did Doi stop being president?
...,...,...,...
17778,1500,The $2 bill makes up just 3 percent of all pap...,Why is the $2 bill so uncommon?
17779,1500,"Now, it 's about to get its time in the limeli...",How is it going to get time in the limelight?
17780,1500,"Now, it 's about to get its time in the limeli...",Who is Delray Beach and why does he matter?
17781,1500,John Bennardo is crisscrossing the country to ...,Why are $2 bills seen as so much more desirable?


In [17]:
train.to_json('./data/train.json', orient='records')

In [18]:
test.to_json('./data/test.json', orient='records')

In [19]:
validation.to_json('./data/validation.json', orient='records')